# Students@SC Boardgame Tournament Ranker

This Jupyter Notebook will calculate the final rankings for the Students@SC Boardgame Tournament based on the provided configuration and score files. Most of the code powering this notebook can be found in the `boardgame_tournament_scoring` directory.

### Configuration File Specification

The configuration file is a TOML file that provides relevant data on each game in the tournament. At the top of the file, there must be a `config_id` field. This is simply used as a unique ID for the file. Each game in the file must have the following format:

```toml
[game_name]
max_ranking_points = ...
number_of_scores = ...
number_of_players = ...
placement_based = [True|False]
```

The fields for each game are as follows:
* `game_name`: the name (i.e., unique identifier) of the game
* `max_ranking_points`: the maximum number of points that this game can contribute towards a contestant's overall score
* `number_of_scores`: the number of scores that this game will contribute towards a contestant's overall score
* `number_of_players`: the maximum number of players for the game
* `placement_based`: if true, scores for this game should be interpreted as placements (i.e., rankings in a single instance of the game).

### Score File Specification

The score file is a CSV file containing information about the results of games played. Each row in this file represents a single contestant's result for a single instance of a game. There are three columns in this file:
1. The `name` column, which specifies the contestant's name
2. The `game` column, which specifies the game the contestant played
3. The `score` column, which specifies the contestant's score/placement in the game

The only other requirement for the score file is that entries in the `game` column **must** match one of the `game_name` fields in the configuration file. If an entry does not match one of these fields, the row will be treated as if it represents an unknown game, and, thus, it will be dropped.

## Specify Configuration and Score Files

In [1]:
tournament_config_file = "./test.toml"
score_file = "./test.csv"

## Get Game Information from Configuration File

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath("./boardgame_tournament_scoring/"))

In [3]:
import pandas as pd

In [4]:
from boardgame_tournament_scoring import (
    import_games_from_toml,
    split_score_df_by_game,
    get_max_scores_no_placing,
    get_normalized_scores,
    get_final_counted_scores_df,
)

In [5]:
games = import_games_from_toml(tournament_config_file)

Importing Game Data for Test Config!


## Read the Score File

In [6]:
score_df = pd.read_csv(score_file)

## Split Score Data by Game

In [7]:
per_game_dfs = split_score_df_by_game(score_df, games)

## Get the Max Scores for Non-Placement Games

In [8]:
max_scores = get_max_scores_no_placing(per_game_dfs, games)

## Normalize the Scores based on Max Ranking Points

In [9]:
normed_per_game_dfs = get_normalized_scores(per_game_dfs, max_scores, games)

## Collect Scores that Should Be Counted Towards Result

In [10]:
full_counted_df = get_final_counted_scores_df(normed_per_game_dfs, games)

# Winner: Overall Score

In [14]:
overall_score_series = full_counted_df.sum(axis=1)
overall_score_series.sort_values(inplace=True, ascending=False)
display(overall_score_series)

name
Ian Lumsden    37.91400
Person 1       35.59495
dtype: float64

# Winner: Most Games Played

In [16]:
games_played_count = score_df[["name", "score"]].groupby(by="name").count()
display(games_played_count)

,score
name,
Ian Lumsden,8
Person 1,8
